# Co-localization Analysis on Drosophila spreads

This notebook analyses co-localization on Drosophila salivary glands spreads

### Import Packages

In [1]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler_core.preferences as cpprefs
import cellprofiler.modules as cpm
import cellprofiler_core.pipeline as cpp
cpprefs.set_headless()

# Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
from cellprofiler_core.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
from omero.gateway import BlitzGateway

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

from getpass import getpass

### Set Cell Output Directory

In [2]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials and dataset to analyze

In [3]:
HOST = 'omero.mri.cnrs.fr'
conn = BlitzGateway(input("Username: "),
                    getpass("OMERO Password: "),
                    host=HOST, secure=True)
conn.connect()

datasetid = input("Dataset id: ")
dataset = conn.getObject("Dataset", datasetid)
print('Dataset Name: ',dataset.getName())

Username: trainer-1
OMERO Password: ········


True

### Load a cellprofiler pipeline attached to the dataset
Be careful, this script only takes the first pipeline that is attached to a dataset, so best keep only one pipeline in the dataset

In [5]:
anns = dataset.listAnnotations()
for ann in anns:
    if type(ann).__name__ == 'FileAnnotationWrapper':
        pipe_path = os.path.join(new_output_directory, ann.getFileName())
        with open(str(pipe_path), 'wb') as f:
            for chunk in ann.getFileInChunks():
                f.write(chunk)

pipeline = cpp.Pipeline()
pipeline.load(str(pipe_path))

# Remove first 4 modules: Images, Metadata, NamesAndTypes, Groups...
# (replaced by InjectImage module below)
for i in range(4):
    print('Remove module: ', pipeline.modules()[0].module_name)
    pipeline.remove_module(1)
print('Pipeline modules:')
for module in pipeline.modules():
    print(module.module_num, module.module_name)

Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
Pipeline modules:
1 Crop
2 Crop
3 Crop
4 IdentifyPrimaryObjects
5 IdentifySecondaryObjects
6 IdentifyTertiaryObjects
7 MeasureObjectSizeShape
8 MeasureObjectIntensity
9 MeasureTexture
10 MeasureObjectNeighbors
11 MeasureColocalization
12 MeasureImageIntensity
13 MeasureImageQuality
14 CalculateMath
15 ClassifyObjects
16 GrayToColor
17 ExportToSpreadsheet


### Run Cell Profiler Pipeline on the OMERO Images

In [6]:
warnings.filterwarnings('ignore')

image_df = pandas.DataFrame()

for image in dataset.listChildren():

    pixels = image.getPrimaryPixels()
    size_c = image.getSizeC()
    # For each Image in OMERO, we copy pipeline and inject image modules
    pipeline_copy = pipeline.copy()

    channel_names = {'HA': 1,
                     'Pc': 2,
                     }

    for image_name, channel in channel_names.items():

        plane = pixels.getPlane(0, channel, 0)
        # image_name = image.getName()

        inject_image_module = InjectImage(image_name, plane)
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)

    m = pipeline_copy.run()

    # Results obtained as CSV from Cell Profiler
    image_csv = pandas.read_csv(new_output_directory + '/Image.csv')
    image_csv.ImageName = image.getName()
    image_csv.ImageId = image.getId()
    image_df = image_df.append(image_csv)

## Results obtained as CSV from Cell Profiler

### Images (Summary)

In [10]:
image_df.head()

,Classify_Large_NumObjectsPerBin,Classify_Large_PctObjectsPerBin,Classify_Medium_NumObjectsPerBin,Classify_Medium_PctObjectsPerBin,Classify_Small_NumObjectsPerBin,Classify_Small_PctObjectsPerBin,Correlation_Correlation_CropBlue_CropGreen,Correlation_Costes_CropBlue_CropGreen,Correlation_Costes_CropGreen_CropBlue,Correlation_K_CropBlue_CropGreen,...,Texture_Variance_CropBlue_3_02_256,Texture_Variance_CropBlue_3_03_256,Threshold_FinalThreshold_Cells,Threshold_FinalThreshold_Nuclei,Threshold_OrigThreshold_Cells,Threshold_OrigThreshold_Nuclei,Threshold_SumOfEntropies_Cells,Threshold_SumOfEntropies_Nuclei,Threshold_WeightedVariance_Cells,Threshold_WeightedVariance_Nuclei
0,3,8.823529,3,8.823529,9,26.470588,0.526074,0.990753,0.954735,0.943040,...,369.211218,372.200473,0.113534,0.054102,0.113534,0.054102,-13.206373,-12.888235,0.636118,1.264639
0,2,7.407407,2,7.407407,9,33.333333,0.178739,0.621968,0.428216,0.446542,...,550.550634,549.096159,0.086443,0.070660,0.086443,0.070660,-12.715738,-12.687216,0.401940,0.931757
0,0,0.000000,1,5.000000,8,40.000000,0.382287,0.941827,0.944726,0.416789,...,314.672127,315.466506,0.062577,0.050299,0.062577,0.050299,-12.997775,-13.107301,0.285742,1.004763


### Push Results back to OMERO

In [12]:
file_to_upload = str(os.path.join(new_output_directory, 'Image.csv'))
image_df.to_csv(file_to_upload)

# create the original file and file annotation (uploads the file etc.)
namespace = "cellprofiler.analysis"
print("\nCreating an OriginalFile and FileAnnotation")
file_ann = conn.createFileAnnfromLocalFile(file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
print("Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(),
    ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize())
dataset.linkAnnotation(file_ann)


Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135224 , Image.csv Size: 54496

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135225 , Nuclei.csv Size: 207531

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135226 , Cytoplasm.csv Size: 185348

Creating an OriginalFile and FileAnnotation
Attaching FileAnnotation to Dataset:  File ID: 135227 , Cells.csv Size: 187106


#### Close the connection to the OMERO server

In [13]:
conn.close()